# SETUP

# NOTE: Please Run this on  Tesla P100-PCIE [ 16280 Mib ]

In [ ]:
!nvidia-smi

Wed Sep 29 05:46:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -r requirements_kaggle.txt -q

> To speed up the review process , i provided the ***drive id*** of the data i've created from the Train creation folder noteboooks .
---
> I  also add each data drive link in the Readme Pdf file attached with this solution

In [ ]:
!pip install -q gdown

In [ ]:
!gdown --id 1-Q7vaYnSEMwQ3jOrm3KwkcqWNI8iEzXn
!gdown --id 1-cl7HvhPuIywmu8lPaK6B1Qb4M6RCIxb
!gdown --id 1hNRbtcqd9F6stMOK1xAZApDITwAjiSDJ
!gdown --id 1-QCmWsNGREXuWArifN0nD_Sp4hJxf0tu

In [ ]:
!gdown --id 10-rLRxkX5Lyf92oM0xtUz1yaTXN3FBGO
!gdown --id 1-9IVDUWu6lhY-DJnfzrjrJgQkafxKS8f
!gdown --id 1-47L_1NKLeVgW1vWmqXXXCuWZ3gwZWsS
!gdown --id 1-aO4FEtv5CF-ZOcxDSO3jGEzPcIFdxgP

In [ ]:
!gdown --id 1-zEJ24ORxrK-RXxxoj770VNHOnGpB0dv
!gdown --id 1-_GnYjOVN8qUB8k1alLBnf3p8_8-GCJj
!gdown --id 1-8J_xFgI0WKT5UXFnfH4q1KUw_KgNY37
!gdown --id 1-a55a7N6a4SoqolPF_wI4C6Q70u_d7Hj

In [ ]:
!gdown --id 1-kzjYxxLepIC8slA9WryJe3ahShANXzt
!gdown --id 1-_-Ps0FoxEWEVCrdYSrCUTLPAFK5CFcs
!gdown --id 1-BgXQwmXqBuk_P8VtvLfdLqy83dv56Kz
!gdown --id 1-hQGF2TNBbsy3jsGNtndmK55egbdFDjs

# LIBRARIES

In [ ]:
#import necessary dependecies
import os
import numpy as np  
import pandas as pd

import random
from tqdm import tqdm 
import copy

from sklearn.preprocessing import  LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.preprocessing import QuantileTransformer

import warnings
warnings.filterwarnings('ignore') 

# tf 
import tensorflow as tf
from tensorflow.keras.layers import Concatenate , Input ,concatenate ,add
from tensorflow.keras.models import Sequential , Model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout
from tensorflow.keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from tensorflow.keras.preprocessing import sequence, text
from tensorflow.keras.callbacks import EarlyStopping , LearningRateScheduler
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.initializers import glorot_normal , l2
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam

# fix seed
tf.random.set_seed(111)
np.random.seed(111)
random.seed(111)

2021-09-29 05:49:46.178613: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


# Train Creation

In [ ]:
def create_train():
  train =pd.read_csv("S2TrainObs1.csv" )
  trainS1 =pd.read_csv("S1TrainObs1.csv" )

  train = pd.merge(train,trainS1.drop('label',axis=1),on='field_id',how='left')
  train = train.groupby('field_id').median().reset_index().sort_values('field_id')
  train.label = train.label.astype('int')
  return train

In [ ]:
def create_test():
  test =pd.read_csv("S2TestObs1.csv" )
  testS1 =pd.read_csv("S1TestObs1.csv" )
  return  pd.merge(test,testS1,on='field_id',how='left')

In [ ]:
def createObs2_train():
  train =pd.read_csv("S2TrainObs2.csv" )
  trainS1 =pd.read_csv("S1TrainObs2.csv" )

  train = pd.merge(train,trainS1.drop('label',axis=1),on='field_id',how='left')
  train = train.groupby('field_id').median().reset_index().sort_values('field_id')
  train.label = train.label.astype('int')
  return train

In [ ]:
def createObs2_test():
  test =pd.read_csv("S2TestObs2.csv" )
  testS1 =pd.read_csv("S1TestObs2.csv" )
  return pd.merge(test,testS1,on='field_id',how='left')

In [ ]:
def createObs3_train():
  train =pd.read_csv("S2TrainObs3.csv" )
  trainS1 =pd.read_csv("S1TrainObs3.csv" )

  train = pd.merge(train,trainS1.drop('label',axis=1),on='field_id',how='left')
  train = train.groupby('field_id').median().reset_index().sort_values('field_id')
  train.label = train.label.astype('int')
  return train

In [ ]:
def createObs3_test():
  test =pd.read_csv("S2TestObs3.csv" )
  testS1 =pd.read_csv("S1TestObs3.csv" )
  return pd.merge(test,testS1,on='field_id',how='left')

In [ ]:
def createObs4_train():
  train =pd.read_csv("S2TrainObs4.csv" )
  trainS1 =pd.read_csv("S1TrainObs4.csv" )

  train = pd.merge(train,trainS1.drop('label',axis=1),on='field_id',how='left')
  train = train.groupby('field_id').median().reset_index().sort_values('field_id')
  train.label = train.label.astype('int')
  return train

In [ ]:
def createObs4_test():
  test =pd.read_csv("S2TestObs4.csv" )
  testS1 =pd.read_csv("S1TestObs4.csv" )
  return pd.merge(test,testS1,on='field_id',how='left')

# Feature Engineering

In [ ]:
def process(T) :

  # process bands
  Bcols = T.filter(like='B').columns.tolist()
  Vcols = T.filter(like='V').columns.tolist()
  Obs1 = T.filter(like='Month4').columns.tolist()
  Obs2 = T.filter(like='Month5').columns.tolist()
  Obs3 = T.filter(like='Month6').columns.tolist()
  Obs4 = T.filter(like='Month7').columns.tolist()
  Obs5 = T.filter(like='Month8').columns.tolist()
  Obs6 = T.filter(like='Month9').columns.tolist()
  Obs7 = T.filter(like='Month10').columns.tolist()
  Obs8 = T.filter(like='Month11').columns.tolist()


  # vegetation indexes 
  B8cols = T.filter(like='B8_').columns.tolist()
  B8cols = [x for x in B8cols if 'std' not in x]
  
  B4cols = T.filter(like='B4_').columns.tolist()
  B4cols = [x for x in B4cols if 'std' not in x]

  B3cols = T.filter(like='B3_').columns.tolist()
  B3cols = [x for x in B3cols if 'std' not in x]

  B5cols = T.filter(like='B5_').columns.tolist()
  B5cols = [x for x in B5cols if 'std' not in x]

  B3cols = T.filter(like='B3_').columns.tolist()
  B3cols = [x for x in B3cols if 'std' not in x]

  B2cols = T.filter(like='B2_').columns.tolist()
  B2cols = [x for x in B2cols if 'std' not in x]

  B7cols = T.filter(like='B7_').columns.tolist()
  B7cols = [x for x in B7cols if 'std' not in x]

  B8Acols = T.filter(like='B8A_').columns.tolist()
  B8Acols = [x for x in B8Acols if 'std' not in x]

  B6cols = T.filter(like='B6_').columns.tolist()
  B6cols = [x for x in B6cols if 'std' not in x]
  
  B12cols = T.filter(like='B12_').columns.tolist()
  B12cols = [x for x in B12cols if 'std' not in x]

  B11cols = T.filter(like='B11_').columns.tolist()
  B11cols = [x for x in B11cols if 'std' not in x]

  B1cols = T.filter(like='B1_').columns.tolist()
  B1cols = [x for x in B1cols if 'std' not in x]

  B9cols = T.filter(like='B9_').columns.tolist()
  B9cols = [x for x in B9cols if 'std' not in x]

  L = 0.725
  for b1,b2 ,b3 ,b4, b5 , b6, b7, b8 ,b8a ,b9,b11,b12 in zip(B1cols,B2cols,B3cols,B4cols,B5cols,B6cols,B7cols,B8cols,B8Acols,B9cols,B11cols,B12cols) :
    T[f'NDVI_{b8.split("_")[1]}']   = ((T[b8] - T[b4]) /  (T[b8] + T[b4])).values.clip(-5,5)
    T[f'SAVI_{b8.split("_")[1]}']   = ((T[b8] - T[b4]) /  (T[b8] + T[b4]+L) * (1.0 + L)).values.clip(-5,5)
    # T[f'CCCI_{b8.split("_")[1]}']   =  ((T[b8] - T[b5]) /  (T[b8] + T[b5])) / ((T[b8] - T[b4]) /  (T[b8] + T[b4]))
    T[f'GRNDVI_{b8.split("_")[1]}'] = ((T[b8] - (T[b3]+T[b4])) /  (T[b8] + (T[b3]+T[b4]))).values.clip(-5,5)
    T[f'GNDVI_{b8.split("_")[1]}']  = ((T[b8] - T[b3] ) /  (T[b8] + T[b3])).values.clip(-5,5)
    # T[f'WDRVI_{b8.split("_")[1]}']  = (0.1 * T[b5] - T[b4])/ (0.1 * T[b5] + T[b4])
    T[f'NDRE_{b8.split("_")[1]}']   = ((T[b5] - T[b4])/ (T[b5] + T[b4])).values.clip(-5,5)
    T[f'EVI_{b8.split("_")[1]}']    = (2.5 * (T[b8]  - T[b4] ) / ((T[b8]  + 6.0 * T[b4]  - 7.5 * T[b2]) + 1.0)).values.clip(min=-5,max=5)
    T[f'WDRVI_{b8.split("_")[1]}']  = (((8 * T[b8]) - T[b4])/ ((8* T[b8]) + T[b4])).values.clip(-5,5)
    T[f'ExBlue_{b8.split("_")[1]}']  = ((2 * T[b2]) - (T[b3]+T[b4]))
    T[f'ExGreen_{b8.split("_")[1]}']  = ((2 * T[b3]) - (T[b2]+T[b4]) )
    T[f'NDRE7_{b8.split("_")[1]}']   = ((T[b7] - T[b4])/ (T[b7] + T[b4])).values.clip(-5,5)
    T[f'MTCI_{b8.split("_")[1]}']   = ((T[b8a] - T[b6])/ (T[b7] + T[b6])).values.clip(-5,5)
    T[f'VARI_{b8.split("_")[1]}']   = ((T[b3] - T[b4])/ (T[b3] + T[b4] - T[b2])).values.clip(-5,5)
    T[f'b3b1_{b8.split("_")[1]}']  = (T[b3] - T[b1])/ (T[b3] + T[b1])    # B7  / B3
    T[f'b11b8_{b8.split("_")[1]}']  = (T[b11] - T[b8])/ (T[b11] + T[b8])    # B7  / B3
    T[f'b12b11_{b8.split("_")[1]}']  = (T[b12] - T[b11])/ (T[b12] + T[b11])    # B7  / B3
    T[f'b3b4_{b8.split("_")[1]}']  = (T[b3] - T[b4])/ (T[b3] + T[b4])    # B7  / B3
    T[f'b9b4_{b8.split("_")[1]}']  = (T[b9] - T[b4])/ (T[b9] + T[b4])    # B7  / B3
    T[f'b5b3_{b8.split("_")[1]}']  = (T[b5] - T[b3])/ (T[b5] + T[b3])    # B7  / B3
    T[f'b12b3_{b8.split("_")[1]}']  = (T[b12] - T[b3])/ (T[b12] + T[b3])    # B7  / B3
    
    T[f'b2b1_{b8.split("_")[1]}']  = (T[b2] - T[b1])/ (T[b2] + T[b1])    # B7  / B3
    T[f'b4b1_{b8.split("_")[1]}']  = (T[b4] - T[b1])/ (T[b4] + T[b1])    # B7  / B3
    T[f'b11b3_{b8.split("_")[1]}']  = (T[b11] - T[b3])/ (T[b11] + T[b3]) 
    
  for col in Bcols :
    T[col] = np.sqrt(T[col])
  
  for col in Vcols :
    T[col] = np.sqrt(T[col])
    
  for col1,col2,col3,col4,col5,col6,col7,col8 in zip(Obs1,Obs2,Obs3,Obs4,Obs5,Obs6,Obs7,Obs8) :
    T[f'{col1.split("_")[0]}_std'] = T[[col1,col2,col3,col4,col5,col6,col7,col8]].std(axis=1)
    T[f'{col1.split("_")[0]}_mean'] = T[[col1,col2,col3,col4,col5,col6,col7,col8]].mean(axis=1)

    
  # NDVI 
  T['water']       = T['NDVI_Month4'].apply(lambda x :1 if x<0 else 0)
  T['dense_green'] = T['NDVI_Month4'].apply(lambda x :1 if x>=0.5 else 0)
  T['not_green']   = T['NDVI_Month4'].apply(lambda x :1 if( (x>0) & (x<0.5)) else 0)

  T['high_green'] = T['SAVI_Month4'].apply(lambda x :1 if x<0.1 else 0)
  T['low_green']  = T['SAVI_Month4'].apply(lambda x :1 if x>=0.8 else 0)
  T['chlorophyll_EVI'] = T['EVI_Month4'].apply(lambda x :1 if( (x>0.2) & (x<0.8)) else 0)

  # process Vegetation indexes

  ObsN   = T.filter(like='NDVI_').columns.tolist()
  ObsSA  = T.filter(like='SAVI_').columns.tolist()
  ObsCC  = T.filter(like='CCCI_').columns.tolist()
  ObsWDR = T.filter(like='WDRVI_').columns.tolist()
  ObsNDRE7 = T.filter(like='NDRE7_').columns.tolist()

  return T

In [ ]:
Train = create_train()
Test = create_test()

In [ ]:
Train.shape , Test.shape

((87114, 114), (35295, 113))

In [ ]:
Train2 = createObs2_train()
Test2 = createObs2_test()

In [ ]:
Train2.shape , Test2.shape

((87114, 114), (35295, 113))

In [ ]:
Train3 = createObs3_train()
Test3 = createObs3_test()

In [ ]:
Train3.shape , Test3.shape

((87114, 114), (35295, 113))

In [ ]:
Train4 = createObs4_train()
Test4 = createObs4_test()

In [ ]:
Train4.shape , Test4.shape

((87114, 114), (35295, 113))

In [ ]:
Train = process(Train)
Test = process(Test)

In [ ]:
Train.shape , Test.shape

((87114, 324), (35295, 323))

In [ ]:
Train2 = process(Train2)
Test2 = process(Test2)

In [ ]:
Train2.shape , Test2.shape

((87114, 324), (35295, 323))

In [ ]:
Train3 = process(Train3)
Test3 = process(Test3)

In [ ]:
Train3.shape , Test3.shape

((87114, 324), (35295, 323))

In [ ]:
Train4 = process(Train4)
Test4 = process(Test4)

In [ ]:
Train4.shape , Test4.shape

((87114, 324), (35295, 323))

In [ ]:
Train = pd.concat([Train,Train2.drop(columns=['field_id','label']),Train3.drop(columns=['field_id','label']),Train4.drop(columns=['field_id','label'])],axis=1)
Train.shape

(87114, 1290)

In [ ]:
Test = pd.concat([Test,Test2.drop(columns=['field_id']),Test3.drop(columns=['field_id'])],axis=1)
Test = pd.merge(Test,Test4,on='field_id',how='left')
Test.shape

(35295, 1289)

# MODELING

In [ ]:
X    = Train.fillna(-999).replace(np.inf,9999).drop(['field_id','label'], axis=1)
y    = Train.label
TEST = Test.fillna(-999).replace(np.inf,9999).drop(['field_id'], axis=1)

In [ ]:
TEST.columns = X.columns.tolist()

In [ ]:
data = pd.concat([X,TEST])
qt=QuantileTransformer(output_distribution="normal",random_state=42)
data= pd.DataFrame(qt.fit_transform(data),columns=X.columns)

In [ ]:
X.shape , TEST.shape

((87114, 1288), (35295, 1288))

In [ ]:
def attention_3d_block(inputs, name):
  # inputs.shape = (batch_size, time_steps, input_dim)
  TIME_STEPS = inputs.shape[1]
  SINGLE_ATTENTION_VECTOR = False
  input_dim = inputs.shape[2]
  a = Permute((2, 1))(inputs)
  a = Dense(TIME_STEPS, activation='softmax')(a)
  if SINGLE_ATTENTION_VECTOR:  
    a = Lambda(lambda x: K.mean(x, axis=1))(a)
    a = RepeatVector(input_dim)(a)
  a_probs = Permute((2, 1), name=name)(a)
  output_attention_mul = Multiply()([inputs, a_probs])
  return output_attention_mul

In [ ]:
def get_model():
  tf.random.set_seed(1)
  np.random.seed(1)
  random.seed(1)
    
  input_tensor = Input(shape=(1,X.shape[2]))
  
  x = BatchNormalization()(input_tensor)
  x = Conv1D(256, 5,strides=5,padding='same',activation='relu')(x)
  x = attention_3d_block(x, 'attention_vec_1')
  x = Dropout(0.1)(x)
  
  x = BatchNormalization()(x)
  x = Conv1D(128, 7,strides=3,padding='same',activation='relu')(x)
  x = attention_3d_block(x, 'attention_vec_2')
  x = Dropout(0.15)(x)
  
  x = BatchNormalization()(x)
  x = Conv1D(512, 9,strides=3,padding='same',activation='relu')(x)
  x = attention_3d_block(x, 'attention_vec_3')
  x = Dropout(0.15)(x)

  x = GlobalMaxPooling1D()(x)

  x = Dropout(0.3)(x)

  out = Dense(9,kernel_initializer=glorot_normal(seed=1),
                bias_initializer=glorot_normal(seed=1),
                activation="softmax")(x)

  model = Model(inputs=input_tensor,outputs =out)

  model.compile(optimizer='adam', loss='categorical_crossentropy')
  return model

In [ ]:
X = data[:X.shape[0]].values
TEST = data[X.shape[0]:].values

In [ ]:
X = X.reshape(X.shape[0], 1,X.shape[1])
TEST = TEST.reshape(TEST.shape[0],1,TEST.shape[1])

In [ ]:
X.shape , TEST.shape

((87114, 1, 1288), (35295, 1, 1288))

In [ ]:
# Function to seed everything

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

In [ ]:
seed_everything(42)

In [ ]:
y_train = y.copy()
n_labels = y.unique().shape[0]

# we need to binarize the labels for the neural net
LE = LabelEncoder()
ytrain_enc = pd.get_dummies(y_train).values
TARGETS = pd.get_dummies(y_train).columns

y_oof = np.zeros([X.shape[0], n_labels])
y_test = np.zeros([TEST.shape[0], n_labels])

i = 0
metrics = list()
apply_aug = False

In [ ]:
seed_everything(42)
n_splits = 10
kf = StratifiedKFold(n_splits=n_splits, random_state=47, shuffle=True)

for idx , (tr_idx, val_idx) in enumerate(kf.split(X, y)):
    # Verbosity
    VERBOSE = 0
    seed_everything(42)
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',
                                                      mode = 'min',
                                                      patience = 35,
                                                      restore_best_weights = True,
                                                      verbose = 1)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(f'Radiant_NN_{idx}.h5',
                                                    monitor = 'val_loss',
                                                    verbose = 1,
                                                    save_best_only = True,
                                                    save_weights_only = True)

    X_tr, X_vl = X[tr_idx, :], X[val_idx, :]
    y_tr, y_vl = ytrain_enc[tr_idx], ytrain_enc[val_idx]
    y_train_, y_vld_ = y[tr_idx], y[val_idx]

    # 1 CNN ANN
    seed_everything(seed=1)
    model = get_model()
    model.fit( 
        X_tr,
        y_tr,
        validation_data =(X_vl, y_vl),
        callbacks = [early_stopping,  checkpoint],
        epochs = 300,
        batch_size = 64)
    
    
    
    
    model = get_model()
    model.load_weights(f'Radiant_NN_{idx}.h5')
    y_pred_cnn_nn = model.predict(X_vl)
    test_pred_cnn_nn = model.predict(TEST, batch_size= 1024)
    
    
    y_oof[val_idx, :] = y_pred_cnn_nn
    y_tr, y_vl = y_train.iloc[tr_idx], y_train.iloc[val_idx]
    metric = log_loss(y_vl, y_pred_cnn_nn)
    print("fold #{} Log Loss: {}".format(i, metric))
    
    i += 1
    test_pred = test_pred_cnn_nn 
    y_test += test_pred / n_splits
    metrics.append(metric)
    
metrics = np.array(metrics).mean()
print(f'Full Log loss {metrics}') 

2021-09-29 05:51:31.921587: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-29 05:51:31.924888: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-29 05:51:31.965828: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-29 05:51:31.966486: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-09-29 05:51:31.966562: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-09-29 05:51:31.994342: I tensorflow/stream_executor/platform/def

Epoch 1/300


2021-09-29 05:51:35.981193: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-09-29 05:51:36.782904: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2021-09-29 05:51:36.815558: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


1226/1226 [==============================] - 20s 10ms/step - loss: 1.1433 - val_loss: 0.9191

Epoch 00001: val_loss improved from inf to 0.91915, saving model to Radiant_NN_0.h5
Epoch 2/300
1226/1226 [==============================] - 12s 9ms/step - loss: 0.9250 - val_loss: 0.8633

Epoch 00002: val_loss improved from 0.91915 to 0.86328, saving model to Radiant_NN_0.h5
Epoch 3/300
1226/1226 [==============================] - 12s 9ms/step - loss: 0.8660 - val_loss: 0.8275

Epoch 00003: val_loss improved from 0.86328 to 0.82749, saving model to Radiant_NN_0.h5
Epoch 4/300
1226/1226 [==============================] - 12s 10ms/step - loss: 0.8320 - val_loss: 0.8005

Epoch 00004: val_loss improved from 0.82749 to 0.80048, saving model to Radiant_NN_0.h5
Epoch 5/300
1226/1226 [==============================] - 12s 10ms/step - loss: 0.8027 - val_loss: 0.7873

Epoch 00005: val_loss improved from 0.80048 to 0.78732, saving model to Radiant_NN_0.h5
Epoch 6/300
1226/1226 [=========================

In [ ]:
print('NN LOG LOSS :',log_loss(y_train,y_oof)) 

NN LOG LOSS : 0.6705215626569802


In [ ]:
# In this part we format the DataFrame to have column names and order similar to the sample submission file. 
pred_df = pd.DataFrame(y_test)
pred_df = pred_df.rename(columns={
    0:'Crop_ID_1',
    1:'Crop_ID_2', 
    2:'Crop_ID_3',
    3:'Crop_ID_4',
    4:'Crop_ID_5',
    5:'Crop_ID_6',
    6:'Crop_ID_7',
    7:'Crop_ID_8',
    8:'Crop_ID_9'
})
pred_df['field_id'] = Test['field_id'].astype('int').values
pred_df = pred_df[['field_id', 'Crop_ID_1', 'Crop_ID_2', 'Crop_ID_3', 'Crop_ID_4', 'Crop_ID_5', 'Crop_ID_6', 'Crop_ID_7', 'Crop_ID_8', 'Crop_ID_9']]
pred_df.head()

,field_id,Crop_ID_1,Crop_ID_2,Crop_ID_3,Crop_ID_4,Crop_ID_5,Crop_ID_6,Crop_ID_7,Crop_ID_8,Crop_ID_9
0,30,0.055811,0.297380,0.011131,0.000204,0.173487,2.992985e-01,1.538131e-01,7.317784e-03,1.557601e-03
1,39,0.768878,0.168540,0.015459,0.015719,0.021324,9.927223e-03,1.141302e-04,2.858439e-05,1.057026e-05
2,49,0.008941,0.397148,0.000055,0.000038,0.000510,5.465559e-01,3.995302e-02,6.780147e-03,1.868593e-05
3,54,0.000001,0.000034,0.000092,0.999768,0.000104,5.213484e-07,3.247506e-07,1.512979e-10,2.743625e-08
4,56,0.884511,0.015084,0.000476,0.000054,0.000644,2.719114e-02,6.365343e-02,8.380454e-03,6.204341e-06


In [ ]:
# Write the predicted probabilites to a csv for submission
pred_df.to_csv('S1_NNAttention.csv', index=False)

In [ ]:
np.save('S1_oof_NNAttention.npy',y_oof)